# UNRAVEL: Velocity Dealiasing Tutorial

This notebook demonstrates how to use UNRAVEL for Doppler velocity dealiasing with both PyART and PyODIM data formats.

## What is UNRAVEL?

UNRAVEL is a 3D Doppler velocity dealiasing algorithm that:
- Works with both PyART and PyODIM/ODIM H5 formats
- Handles variable Nyquist velocities across sweeps
- Supports preprocessing workflows (e.g., dual-PRF correction)
- Provides multiple dealiasing strategies (default and long-range)

## Installation

```bash
# Core installation
pip install unravel-radar

# With PyART support
pip install unravel-radar[pyart]

# With ODIM support
pip install unravel-radar[odim]

# With everything
pip install unravel-radar[all]
```

## About the Example Data

This tutorial uses:
- **PyART Example**: CPOL (C-band Polarimetric) radar from Darwin, Australia
  - Available dates: December 1998 - May 2017 (wet season only: Nov-Apr)
  - We'll download data from February 18, 2014
- **PyODIM Example**: ODIM H5 format radar data
  - Uses local file: `data/49_20240825_070000.pvol.h5`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Libraries imported successfully!")

# Example 1: PyART Workflow

Using UNRAVEL with PyART radar objects.

## Download CPOL radar data

We'll use data from the C-band Polarimetric (CPOL) radar in Darwin, Australia.

In [ ]:
import os
import datetime
import requests

def download_cpol_data(date: datetime.datetime) -> str:
    """
    Download CPOL data for given date.
    
    Parameters:
    ===========
    date: datetime.datetime
        Date time for which we want the CPOL data
    
    Returns:
    ========
    str: Path to downloaded file
    """
    year = date.year
    datestr = date.strftime("%Y%m%d")
    datetimestr = date.strftime("%Y%m%d.%H%M")
    
    url = f"https://dapds00.nci.org.au/thredds/fileServer/hj10/cpol/cpol_level_1b/v2020/ppi/{year}/{datestr}/twp10cpolppi.b1.{datetimestr}00.nc"
    
    fname = os.path.basename(url)
    
    # Create download directory
    try:
        os.mkdir("data")
    except FileExistsError:
        pass
    
    outfilename = os.path.join("data", fname)
    
    if os.path.isfile(outfilename):
        print(f"File already exists: {outfilename}")
        return outfilename
    
    print(f"Downloading CPOL data from {date}...")
    r = requests.get(url)
    
    try:
        r.raise_for_status()
    except Exception:
        raise ValueError(
            "No file found for this date. CPOL ran from 1998-12-6 to 2017-5-2, "
            "wet season only (Nov-Apr). Try another date."
        )
    
    with open(outfilename, "wb") as fid:
        fid.write(r.content)
    
    print(f"Download complete: {outfilename}")
    return outfilename

# Download data from February 18, 2014 at 20:00 UTC
# This is a known good case with interesting velocity patterns
date = datetime.datetime(2014, 2, 18, 20, 0)

# Try other dates during CPOL operation (Dec 1998 - May 2017, wet season only)
# Example dates:
# date = datetime.datetime(2006, 1, 15, 12, 0)  # January 2006
# date = datetime.datetime(2010, 3, 20, 18, 0)  # March 2010

radar_file = download_cpol_data(date)
print(f"\nRadar file ready: {radar_file}")

## Read radar data with PyART

In [ ]:
import pyart
import unravel

# Read radar data
radar = pyart.io.read(radar_file)

print(f"Radar: {radar.metadata.get('instrument_name', 'CPOL')}")
print(f"Number of sweeps: {radar.nsweeps}")
print(f"Available fields: {list(radar.fields.keys())}")
print(f"\nSweep elevations: {[f'{radar.fixed_angle["data"][i]:.1f}°' for i in range(radar.nsweeps)]}")

## Dealias the velocity field

CPOL uses specific field names:
- `velocity`: Doppler velocity
- `corrected_reflectivity`: Corrected reflectivity (used for filtering)

In [ ]:
# Run dealiasing
dealiased_velocity = unravel.unravel_3D_pyart(
    radar,
    velname='velocity',                    # CPOL velocity field name
    dbzname='corrected_reflectivity',      # CPOL reflectivity field name
    nyquist_velocity=13.3,                 # CPOL Nyquist velocity (m/s)
    strategy='default',                    # or 'long_range'
    alpha=0.8,                             # Threshold parameter
    do_3d=True                             # Enable 3D continuity
)

print(f"Dealiased velocity shape: {dealiased_velocity.shape}")
print(f"Original velocity shape: {radar.fields['velocity']['data'].shape}")
print(f"\nDealiasing completed successfully!")

## Add dealiased field to radar object

In [ ]:
# Create field dictionary
vel_dict = {
    'data': dealiased_velocity,
    'units': 'm/s',
    'long_name': 'Dealiased Doppler velocity',
    'standard_name': 'radial_velocity_of_scatterers_away_from_instrument',
    'valid_min': -100.0,
    'valid_max': 100.0,
}

# Add to radar object
radar.add_field('velocity_dealiased', vel_dict, replace_existing=True)
print("Dealiased velocity field added to radar object")

## Visualize results: Before and After

In [ ]:
def plot_ppi_comparison(radar, sweep=0, vmin=-30, vmax=30):
    """
    Plot side-by-side comparison of original and dealiased velocity.
    """
    fig = plt.figure(figsize=(12, 5.5), constrained_layout=True)
    gs = GridSpec(1, 3, width_ratios=[1, 1, 0.05], figure=fig)
    
    # Original velocity
    ax1 = fig.add_subplot(gs[0])
    display = pyart.graph.RadarDisplay(radar)
    display.plot_ppi(
        'velocity',
        sweep=sweep,
        ax=ax1,
        vmin=vmin,
        vmax=vmax,
        cmap='BuDRd18',
        title=f'Original Velocity - Sweep {sweep}',
        colorbar_flag=False
    )
    
    # Dealiased velocity
    ax2 = fig.add_subplot(gs[1])
    display.plot_ppi(
        'velocity_dealiased',
        sweep=sweep,
        ax=ax2,
        vmin=vmin,
        vmax=vmax,
        cmap='BuDRd18',
        title=f'Dealiased Velocity - Sweep {sweep}',
        colorbar_flag=False
    )
    
    # Shared colorbar
    cax = fig.add_subplot(gs[2])
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    cb = plt.colorbar(
        plt.cm.ScalarMappable(norm=norm, cmap='BuDRd18'),
        cax=cax,
        label='Velocity (m/s)'
    )
    
    
    return fig

# Plot for first sweep
fig = plot_ppi_comparison(radar, sweep=0, vmin=-40, vmax=40)
plt.show()

# Example 2: PyODIM Workflow

Using UNRAVEL with ODIM H5 files.

In [ ]:
import pyodim
import xarray as xr

# Option 1: Direct file reading (simplest)
odim_file = '../tests/data/49_20240825_070000.pvol.h5'

datasets = unravel.unravel_3D_pyodim(
    odim_file,
    vel_name='VRADH',
    output_vel_name='velocity_dealiased',
    strategy='long_range',
    alpha=0.6
)

print(f"Number of sweeps processed: {len(datasets)}")
print(f"First sweep fields: {list(datasets[0].data_vars)}")

## Option 2: Preprocessing workflow

For users who need to apply corrections before dealiasing (e.g., dual-PRF correction).

In [ ]:
# Step 1: Load data with pyodim
datasets_raw = pyodim.read_odim(odim_file, lazy_load=False)
datasets_raw = [ds.compute() for ds in datasets_raw]

print(f"Loaded {len(datasets_raw)} sweeps")

# Step 2: Apply preprocessing (example: dual-PRF correction)
# datasets_preprocessed = apply_dualprf_correction(datasets_raw)  # Your preprocessing here
datasets_preprocessed = datasets_raw  # For demo, no preprocessing

# Step 3: Dealias the preprocessed data
datasets_dealiased = unravel.unravel_3D_pyodim(
    datasets_preprocessed,  # Pass pre-loaded datasets
    vel_name='VRADH',
    output_vel_name='velocity_dealiased',
    strategy='long_range',
    alpha=0.6
)

print("Preprocessing + dealiasing completed!")

## Visualize ODIM results

In [ ]:
def plot_odim_ppi(dataset, field_original='VRADH', field_dealiased='velocity_dealiased', 
                  vmin=-30, vmax=30):
    """
    Plot PPI comparison for ODIM dataset.
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 6), constrained_layout=True)
    
    # Get coordinates
    azimuth = dataset.azimuth.values
    range_vals = dataset.range.values
    
    # Convert to Cartesian coordinates for plotting
    az_rad = np.deg2rad(azimuth)
    R, AZ = np.meshgrid(range_vals, az_rad)
    X = R * np.sin(AZ)
    Y = R * np.cos(AZ)
    
    # Original velocity
    vel_orig = dataset[field_original].values
    im1 = axes[0].pcolormesh(
        X/1000, Y/1000, vel_orig,
        cmap='RdBu_r',
        vmin=vmin, vmax=vmax,
        shading='auto'
    )
    axes[0].set_title(f'Original Velocity\nElevation: {dataset.elevation.values[0]:.1f}°')
    axes[0].set_xlabel('East-West Distance (km)')
    axes[0].set_ylabel('North-South Distance (km)')
    axes[0].set_aspect('equal')
    axes[0].grid(True, alpha=0.3)
    
    # Dealiased velocity
    vel_dealias = dataset[field_dealiased].values
    im2 = axes[1].pcolormesh(
        X/1000, Y/1000, vel_dealias,
        cmap='RdBu_r',
        vmin=vmin, vmax=vmax,
        shading='auto'
    )
    axes[1].set_title(f'Dealiased Velocity\nNyquist: {dataset.attrs["NI"]:.1f} m/s')
    axes[1].set_xlabel('East-West Distance (km)')
    axes[1].set_ylabel('North-South Distance (km)')
    axes[1].set_aspect('equal')
    axes[1].grid(True, alpha=0.3)
    
    # Colorbar
    fig.colorbar(im2, ax=axes, label='Velocity (m/s)', fraction=0.046, pad=0.04)
    
    return fig

# Plot first sweep
fig = plot_odim_ppi(datasets_dealiased[0], vmin=-40, vmax=40)
plt.show()

## Plot all sweeps

In [ ]:
def plot_all_sweeps(datasets, field='velocity_dealiased', vmin=-30, vmax=30):
    """
    Plot all sweeps in a grid.
    """
    n_sweeps = len(datasets)
    n_cols = 3
    n_rows = int(np.ceil(n_sweeps / n_cols))
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4.25*n_rows), constrained_layout=True)
    axes = axes.flatten() if n_sweeps > 1 else [axes]
    
    for idx, ds in enumerate(datasets):
        ax = axes[idx]
        
        # Convert to Cartesian
        azimuth = ds.azimuth.values
        range_vals = ds.range.values
        az_rad = np.deg2rad(azimuth)
        R, AZ = np.meshgrid(range_vals, az_rad)
        X = R * np.sin(AZ)
        Y = R * np.cos(AZ)
        
        # Plot
        vel = ds[field].values
        im = ax.pcolormesh(
            X/1000, Y/1000, vel,
            cmap='RdBu_r',
            vmin=vmin, vmax=vmax,
            shading='auto'
        )
        
        elev = ds.elevation.values[0]
        nyq = ds.attrs['NI']
        ax.set_title(f'Sweep {idx} - Elev: {elev:.1f}° - Nyq: {nyq:.1f} m/s')
        ax.set_xlabel('X (km)')
        ax.set_ylabel('Y (km)')
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)
        
        plt.colorbar(im, ax=ax, label='Velocity (m/s)')
    
    # Hide extra subplots
    for idx in range(n_sweeps, len(axes)):
        axes[idx].axis('off')
    
    return fig

# Plot all sweeps
fig = plot_all_sweeps(datasets_dealiased, vmin=-40, vmax=40)
plt.show()

# Advanced: Quality Assessment

Check the dealiasing quality using the flag field.

In [ ]:
def plot_quality_flags(dataset, flag_field='velocity_dealiased_flag'):
    """
    Plot the dealiasing quality flags.
    
    Flag values:
    -3: No data
     0: Unprocessed
     1: Processed (no change needed)
     2: Processed (dealiased)
    """
    fig, ax = plt.subplots(figsize=(10, 8), constrained_layout=True)
    
    # Convert to Cartesian
    azimuth = dataset.azimuth.values
    range_vals = dataset.range.values
    az_rad = np.deg2rad(azimuth)
    R, AZ = np.meshgrid(range_vals, az_rad)
    X = R * np.sin(AZ)
    Y = R * np.cos(AZ)
    
    flags = dataset[flag_field].values
    
    # Custom colormap for flags
    cmap = colors.ListedColormap(['gray', 'yellow', 'lightgreen', 'blue'])
    bounds = [-3.5, -0.5, 0.5, 1.5, 2.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)
    
    im = ax.pcolormesh(
        X/1000, Y/1000, flags,
        cmap=cmap, norm=norm,
        shading='auto'
    )
    
    ax.set_title('Dealiasing Quality Flags')
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    
    # Custom colorbar
    cbar = plt.colorbar(im, ax=ax, ticks=[-3, 0, 1, 2])
    cbar.ax.set_yticklabels(['No Data', 'Unprocessed', 'No Change', 'Dealiased'])
    cbar.set_label('Processing Status')
    
    # Calculate statistics
    total_gates = flags.size
    no_data = np.sum(flags == -3)
    unprocessed = np.sum(flags == 0)
    no_change = np.sum(flags == 1)
    dealiased = np.sum(flags == 2)
    
    stats_text = f"""Statistics:
    Total gates: {total_gates}
    No data: {no_data} ({100*no_data/total_gates:.1f}%)
    Unprocessed: {unprocessed} ({100*unprocessed/total_gates:.1f}%)
    No change: {no_change} ({100*no_change/total_gates:.1f}%)
    Dealiased: {dealiased} ({100*dealiased/total_gates:.1f}%)
    """
    
    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
            fontfamily='monospace', fontsize=9)
    
    return fig

# Plot quality flags for first sweep
fig = plot_quality_flags(datasets_dealiased[0])
plt.show()

# Strategy Comparison

Compare 'default' vs 'long_range' strategies.

In [ ]:
# Run both strategies
datasets_default = unravel.unravel_3D_pyodim(
    odim_file,
    vel_name='VRADH',
    output_vel_name='vel_default',
    strategy='default',
    alpha=0.6
)

datasets_longrange = unravel.unravel_3D_pyodim(
    odim_file,
    vel_name='VRADH',
    output_vel_name='vel_longrange',
    strategy='long_range',
    alpha=0.6
)

# Plot comparison for first sweep
fig, axes = plt.subplots(1, 3, figsize=(20, 6), constrained_layout=True)

ds_idx = 0
azimuth = datasets_default[ds_idx].azimuth.values
range_vals = datasets_default[ds_idx].range.values
az_rad = np.deg2rad(azimuth)
R, AZ = np.meshgrid(range_vals, az_rad)
X = R * np.sin(AZ)
Y = R * np.cos(AZ)

vmin, vmax = -40, 40

# Original
im0 = axes[0].pcolormesh(
    X/1000, Y/1000, datasets_default[ds_idx]['VRADH'].values,
    cmap='RdBu_r', vmin=vmin, vmax=vmax, shading='auto'
)
axes[0].set_title('Original')
axes[0].set_aspect('equal')

# Default strategy
im1 = axes[1].pcolormesh(
    X/1000, Y/1000, datasets_default[ds_idx]['vel_default'].values,
    cmap='RdBu_r', vmin=vmin, vmax=vmax, shading='auto'
)
axes[1].set_title('Default Strategy')
axes[1].set_aspect('equal')

# Long-range strategy
im2 = axes[2].pcolormesh(
    X/1000, Y/1000, datasets_longrange[ds_idx]['vel_longrange'].values,
    cmap='RdBu_r', vmin=vmin, vmax=vmax, shading='auto'
)
axes[2].set_title('Long-Range Strategy')
axes[2].set_aspect('equal')

for ax in axes:
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.grid(True, alpha=0.3)

fig.colorbar(im2, ax=axes, label='Velocity (m/s)', fraction=0.046, pad=0.04)
plt.show()

# Summary

## Key Parameters

- **`strategy`**: Choose 'default' for typical cases, 'long_range' for data with more extensive aliasing
- **`alpha`**: Threshold parameter (0.6-0.8). Lower values are more aggressive in dealiasing
- **`do_3d`**: Enable 3D continuity check (recommended for volume scans)
- **`nyquist_velocity`**: Can be scalar (same for all sweeps) or list (different per sweep)

## When to Use Each Strategy

- **Default**: Standard weather radar applications, typical velocity ranges
- **Long-range**: Strong winds, tropical cyclones, or when you see significant aliasing

## Tips

1. Always check the quality flags to assess dealiasing performance
2. Compare original and dealiased fields visually
3. For ODIM workflows, use pre-loaded datasets if you need to apply corrections first
4. Adjust `alpha` if you see over/under-dealiasing

## Further Reading

- **UNRAVEL Paper**: Louf, V., A. Protat, R. A. Warren, S. M. Collis, D. B. Wolff, S. Raunyiar, C. Jakob, and W. A. Petersen, 2020: An Integrated Approach to Weather Radar Calibration and Monitoring Using Ground Clutter and Satellite Comparisons. *J. Atmos. Oceanic Technol.*, **37**, 5, 899-916. [https://journals.ametsoc.org/view/journals/atot/37/5/jtech-d-19-0020.1.xml](https://journals.ametsoc.org/view/journals/atot/37/5/jtech-d-19-0020.1.xml)
- **PyART Documentation**: [https://arm-doe.github.io/pyart/](https://arm-doe.github.io/pyart/)
- **ODIM H5 Format Specification**: [OPERA ODIM H5 Documentation](https://github.com/baltrad/hlhdf)